<a href="https://colab.research.google.com/github/beyzaerdogn/ATM/blob/master/notebook9d156d09ed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'playground-series-s4e8:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F76727%2F9045607%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T080002Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D83f6ab0a28cc67784fa2f11054ac11d64aff44427996f488d09d49010efcd789a4b84c1f87ca041037af99361240a8a99d242afb107cffafba1203ce22adcbefe1069ad7ab18f869ad2ad9b684d34e588414f04bff6d3c0cdca114916c63952ccb00403ae4ecdd754beaee16a798ccc9054f64f9cc9d21d9633dbb71f2e40dc3d8e87fa71de2f7cbeaa319ddc3f6f07310c08f674fa89c547b7cc85f51b29c763d39aac39e87bbdfa4bef5c6688a9cbc377cb4f811fbc48547f10d84c74932c1c4b18c0e2f2f7cc4e458fc8827927be018fbeb3d3796f0f3c187efd185584460066a0a9b220a92f70538cf745d52ffc8952f84447484e8758ccf92946f3a2a19'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 86301661 bytes downloaded
Downloaded and uncompressed: playground-series-s4e8
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e8/test.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/sample_submission.csv


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


In [4]:
import numpy as np
import pandas as pd

# Veriyi okuma
train_data = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
test_data = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')



In [5]:
# Sayısal sütunları ortalama ile dolduralım
numerical_columns = train_data.select_dtypes(include=['float64', 'int64']).columns
train_data[numerical_columns] = train_data[numerical_columns].fillna(train_data[numerical_columns].mean())
test_data[numerical_columns] = test_data[numerical_columns].fillna(test_data[numerical_columns].mean())


In [6]:
## Kategorik sütunları belirleyelim, 'class' sütununu hariç tutalım
categorical_columns = train_data.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.drop('class')

# Kategorik sütunları en sık görülen değerle dolduralım
train_data[categorical_columns] = train_data[categorical_columns].apply(lambda x: x.fillna(x.mode()[0]))
test_data[categorical_columns] = test_data[categorical_columns].apply(lambda x: x.fillna(x.mode()[0]))

print(train_data.isnull().sum())
print(test_data.isnull().sum())

id                      0
class                   0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0
has-ring                0
ring-type               0
spore-print-color       0
habitat                 0
season                  0
dtype: int64
id                      0
cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-root               0
stem-surface            0
stem-color              0
veil-type               0
veil-color              0

In [7]:
from sklearn.preprocessing import LabelEncoder
# Ortak kategorik sütunlar
common_categorical_columns = [
    'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed',
    'gill-attachment', 'gill-spacing', 'gill-color', 'stem-root',
    'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring',
    'ring-type', 'spore-print-color', 'habitat', 'season'
]

# One-hot encoding uygulayalım
train_data_encoded = pd.get_dummies(train_data, columns=common_categorical_columns)
test_data_encoded = pd.get_dummies(test_data, columns=common_categorical_columns)


In [8]:
# Eğitim ve test setindeki tüm sütunların aynı olduğundan emin olalım
train_data_encoded, test_data_encoded = train_data_encoded.align(test_data_encoded, join='inner', axis=1)

# Hedef değişkeni ayıralım
label_encoder = LabelEncoder()

# Hedef değişkenin adı 'target' yerine, 'class' olduğunu varsayalım
train_data_encoded['class'] = label_encoder.fit_transform(train_data['class'])

In [9]:

# Özellikler ve hedef değişkeni ayıralım
X_train = train_data_encoded.drop(['id', 'class'], axis=1).values
y_train = train_data_encoded['class'].values

# Test verisini hazırlayalım
X_test = test_data_encoded.drop('id', axis=1).values

In [12]:
X_train = X_train.astype('float32')
y_train = y_train.astype('int32')  # Eğer sınıf etiketleri tamsayılar


In [10]:
num_classes = len(set(y_train))  # Kategorik sınıf sayısı

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # 'sparse_categorical_crossentropy' kullanımı uygun olabilir
              metrics=['accuracy'])


In [13]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)  # %20 veriyi doğrulama için ayırdık


Epoch 1/10
77924/77924 [==============================] - 162s 2ms/step - loss: 0.0514 - accuracy: 0.9878 - val_loss: 0.0438 - val_accuracy: 0.9904
Epoch 2/10
77924/77924 [==============================] - 155s 2ms/step - loss: 0.0436 - accuracy: 0.9903 - val_loss: 0.0431 - val_accuracy: 0.9904
Epoch 3/10
77924/77924 [==============================] - 152s 2ms/step - loss: 0.0426 - accuracy: 0.9906 - val_loss: 0.0414 - val_accuracy: 0.9908
Epoch 4/10
77924/77924 [==============================] - 150s 2ms/step - loss: 0.0421 - accuracy: 0.9907 - val_loss: 0.0415 - val_accuracy: 0.9908
Epoch 5/10
77924/77924 [==============================] - 152s 2ms/step - loss: 0.0418 - accuracy: 0.9908 - val_loss: 0.0409 - val_accuracy: 0.9910
Epoch 6/10
77924/77924 [==============================] - 148s 2ms/step - loss: 0.0417 - accuracy: 0.9909 - val_loss: 0.0416 - val_accuracy: 0.9909
Epoch 7/10
77924/77924 [==============================] - 148s 2ms/step - loss: 0.0417 - accuracy: 0.9909 - val_

In [30]:
# Test verisi üzerinde tahminler yapın
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Kategorik tahminler

# Sonuçları bir DataFrame'e dönüştür
submission_df = pd.DataFrame({'id': test_data['id'], 'class': y_pred_classes})

# Dosyayı kaydedin
submission_df.to_csv('submission1.csv', index=False)


64937/64937 [==============================] - 76s 1ms/step


In [22]:
# Test verisi üzerinde tahminler
X_test = X_test.astype('float32')  # Test verisini de uygun türde dönüştür

# NaN ve sonsuz değerleri kontrol et
X_test = np.nan_to_num(X_test)

# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Kategorik tahminler

# Sonuçları bir DataFrame'e dönüştür
submission = pd.DataFrame({
    'id': test_data['id'],
    'class': y_pred_classes
})

# Sonuçları CSV dosyasına kaydet
submission.to_csv('submission.csv', index=False)


64937/64937 [==============================] - 79s 1ms/step


In [26]:
print(train_data['class'].unique())


['e' 'p']
